#### Funding

In [ ]:
def get_thresholds():
    data = []
    with open('./bot/config/thresholds.txt', 'r') as file:
        for line in file:
            line = line.strip()  # Удаляем пробелы и переносы строк
            if line:  # Игнорируем пустые строки
                # Преобразуем строку в кортеж с помощью literal_eval
                tuple_data = ast.literal_eval(line)
                data.append(tuple_data)
    return data

In [ ]:
from bot.core.exchange.http_api import ExchangeManager, BybitRestAPI, GateIORestAPI

import pandas as pd
import polars as pl
import numpy as np
# pd.options.display.float_format = '{:.2f}'.format
from datetime import datetime, timezone, timedelta

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

import warnings
warnings.filterwarnings("ignore")

from tqdm.notebook import tqdm

from bot.utils.files import get_saved_coins

In [ ]:
async def get_data(symbol, interval, n_iters, exchange):
    df = await exc_manager.get_candles(symbol=symbol, interval=interval, n_iters=n_iters)
    df = df[f'{exchange}_linear'].sort_index()
    return df.dropna()

In [ ]:
exc_manager = ExchangeManager()
exc_manager.add_market("bybit_linear", BybitRestAPI('linear'))
coin_information = exc_manager.get_instrument_data()

In [ ]:
exchange = 'bybit'

token_list = [
            '1INCH_USDT',
            'ACH_USDT', 'ADA_USDT', 'AGLD_USDT', 'ALGO_USDT', 'APT_USDT', 'ARB_USDT', 'ARKM_USDT', 'ATOM_USDT',
            'BLUR_USDT',
            'CELO_USDT', 'CHZ_USDT', 'CRV_USDT',
            'DOGE_USDT', 'DOT_USDT', 'DRIFT_USDT', 'DYDX_USDT',
            'EGLD_USDT', 'ENA_USDT', 'ETHFI_USDT', 'FARTCOIN_USDT', 'FLOW_USDT',
            'GALA_USDT', 'GLM_USDT', 'GMT_USDT', 'GRT_USDT',
            'HBAR_USDT', 'IMX_USDT', 'INJ_USDT', 'IOTA_USDT',
            'JASMY_USDT', 'JUP_USDT', 'KAS_USDT', 'KSM_USDT',
            'LDO_USDT', 'LPT_USDT',
            'MANA_USDT', 'MANTA_USDT', 'MORPHO_USDT', 'MOVE_USDT',
            'NEAR_USDT', 'NXPC_USDT',
            'ONDO_USDT', 'OP_USDT', 'ORDI_USDT',
            'PNUT_USDT', 'POL_USDT', 'RENDER_USDT', 'ROSE_USDT', 'RUNE_USDT',
            'S_USDT', 'SAND_USDT', 'SEI_USDT', 'STRK_USDT',
            'STX_USDT', 'SUI_USDT', 'SUSHI_USDT',
            'THETA_USDT', 'TIA_USDT', 'TON_USDT', 'TRX_USDT', 'VET_USDT',
            'WIF_USDT', 'XLM_USDT', 'XRP_USDT', 'ZRX_USDT'
              ]

syms = [col[:-5] for col in token_list]

In [ ]:
exc_manager = ExchangeManager()
exc_manager.add_market("bybit_linear", BybitRestAPI('linear'))

start_date = datetime.now().replace(microsecond=0) - timedelta(days=60)
end_date = None

for symbol in syms:
    fund_hist = await exc_manager.get_funding_history(symbol=f'{symbol}_USDT', start_date=start_date, limit=500)
    fund_hist = fund_hist[0]

    fund_sum = float(fund_hist['funding'].sum())
    fund_len = fund_hist['funding'].shape[0]

    hour_df = await get_data(symbol=f'{symbol}_USDT', interval='1h', n_iters=5, exchange=exchange)
    hour_df = hour_df.tail(4320) # Берём последние пол года
    hour_df.drop(['Exchange', 'Market_type'], axis=1, inplace=True)
    daily_df = await get_data(symbol=f'{symbol}_USDT', interval='1d', n_iters=1, exchange=exchange)
    daily_df = daily_df.tail(180) # Берём последние пол года

    hour_df['Max_range'] = hour_df['High'] - hour_df['Low']
    hour_df['Max_range_perc'] = (hour_df['High'] - hour_df['Low']) / hour_df['Low'] * 100
    hour_10 = hour_df[hour_df['Max_range_perc'] > 10].shape[0]
    hour_10_perc = 100 * hour_10 / hour_df.shape[0]
    hour_atr = hour_df['Max_range'].mean()
    hour_atr_perc = hour_df['Max_range_perc'].mean()
    hour_mtr_perc = hour_df['Max_range_perc'].max()

    daily_df['Max_range'] = daily_df['High'] - daily_df['Low']
    daily_df['Max_range_perc'] = (daily_df['High'] - daily_df['Low']) / daily_df['Low'] * 100
    days_10 = daily_df[daily_df['Max_range_perc'] > 10].shape[0]
    days_10_perc = 100 * days_10 / daily_df.shape[0]
    day_atr = daily_df['Max_range'].mean()
    day_atr_perc = daily_df['Max_range_perc'].mean()
    day_mtr_perc = daily_df['Max_range_perc'].max()

    print(f'========= {symbol} =========')
    print(f'Суммарный фандинг: {fund_sum * 100:.2f}%, расчётов по фандингу: {fund_len}')
    print(f'Фандинг. min: {fund_hist['funding'].min() * 100}%; max: {fund_hist['funding'].max() * 100}%')
    print()
    print(f'ATR за 1 день: {day_atr:.4f} ({day_atr_perc:.2f}%); за 1 час: {hour_atr:.4f} ({hour_atr_perc:.2f}%)')
    print(f'MaxTR за 1 день: {day_mtr_perc:.2f}%; за 1 час: {hour_mtr_perc:.2f}%')
    print(f'Колебания цены более 10%. Дни: {days_10} ({days_10_perc:.1f}%); Часы: {hour_10} ({hour_10_perc:.1f}%)')
    print()

In [ ]:
daily_df

#### Бектест одной торговой пары

In [1]:
from bot.analysis.backtesting import backtest
from bot.utils.pair_trading import make_df_from_orderbooks, make_trunc_df, create_zscore_df
from bot.utils.pair_trading import get_lr_zscore, get_dist_zscore, get_qty, calculate_profit, calculate_profit_curve
from bot.analysis.strategy_analysis import analyze_strategy
from statsmodels.tsa.stattools import adfuller, coint

from datetime import datetime, timezone, timedelta
from zoneinfo import ZoneInfo
import polars as pl
import numpy as np
import pickle
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['timezone'] = 'Europe/Moscow'

from bot.core.db.postgres_manager import DBManager
from bot.config.credentials import host, user, password, db_name
db_params = {'host': host, 'user': user, 'password': password, 'dbname': db_name}
db_manager = DBManager(db_params)

In [ ]:
token_1 = 'GMT'
token_2 = 'BLUR'
min_order = 42
leverage = 2

valid_length = 5
train_length = 10
pairs = pl.read_parquet('./data/pair_selection/all_pairs.parquet')
try:
    curr_pair = pairs.filter((pl.col('coin1') == token_1) & (pl.col('coin2') == token_2))
    std_1 = pairs['std_1'][0]
    std_2 = pairs['std_2'][0]
except IndexError:
    curr_pair = pairs.filter((pl.col('coin1') == token_2) & (pl.col('coin2') == token_1))
    std_1 = pairs['std_2'][0]
    std_2 = pairs['std_1'][0]

end_time = datetime.now().replace(tzinfo=ZoneInfo("Europe/Moscow"))
valid_time = (end_time - timedelta(days=valid_length)).replace(hour=0, minute=0, second=0, microsecond=0)
start_time = valid_time - timedelta(days=train_length)
start_ts = int(datetime.timestamp(valid_time))

tf = '1h'
winds = np.array([48, 72, 96, 120])

with open("./data/coin_information.pkl", "rb") as f:
    coin_information = pickle.load(f)

dp_1 = float(coin_information['bybit_linear'][token_1 + '_USDT']['qty_step'])
ps_1 = int(coin_information['bybit_linear'][token_1 + '_USDT']['price_scale'])
dp_2 = float(coin_information['bybit_linear'][token_2 + '_USDT']['qty_step'])
ps_2 = int(coin_information['bybit_linear'][token_2 + '_USDT']['price_scale'])

In [ ]:
df_1 = db_manager.get_tick_ob(token=token_1 + '_USDT',
                                     start_time=start_time,
                                     end_time=end_time)
df_2 = db_manager.get_tick_ob(token=token_2 + '_USDT',
                                     start_time=start_time,
                                     end_time=end_time)
print(f'{token_1:>8} first date: {df_1['time'][0]}')
print(f'{token_2:>8} first date: {df_2['time'][0]}')

tick_df = make_df_from_orderbooks(df_1, df_2, token_1, token_2, start_time=start_time)
agg_df = make_trunc_df(tick_df, timeframe=tf, token_1=token_1, token_2=token_2, method='triple')

dist_df = create_zscore_df(token_1, token_2, tick_df, agg_df, tf, winds, min_order, start_ts, 
                             median_length=6, spr_method='dist')
lr_df = create_zscore_df(token_1, token_2, tick_df, agg_df, tf, winds, min_order, start_ts, 
                             median_length=6, spr_method='lr')
tls_df = create_zscore_df(token_1, token_2, tick_df, agg_df, tf, winds, min_order, start_ts, 
                             median_length=6, spr_method='tls')

search_space = [(tf, int(w)) for w in winds]

In [ ]:
fig, ax = plt.subplots(nrows=len(winds), ncols=1, sharex=True, figsize=(15, 3*len(winds)))
for i, w in enumerate(winds):
    ax[i].plot(dist_df['time'], dist_df[f'z_score_{w}_{tf}'], c='blue', label='dist')
    ax[i].plot(lr_df['time'], lr_df[f'z_score_{w}_{tf}'], c='green', label='lr')
    ax[i].axhline(y=0, color='black', linestyle='-', linewidth=2)
    ax[i].axhline(y=-2.5, color='green', linestyle='--', linewidth=1)
    ax[i].axhline(y=2.5, color='green', linestyle='--', linewidth=1)
    ax[i].axhline(y=-0.25, color='red', linestyle='--', linewidth=1)
    ax[i].axhline(y=0.25, color='red', linestyle='--', linewidth=1)
    ax[i].grid()
    ax[i].set_title(f'{tf}; {w}')
    ax[i].legend()
fig.tight_layout()

In [ ]:
coint(agg_df[token_1], agg_df[token_2], trend='c')

In [ ]:
def run_backtest(dist_df, params):
    end_date = dist_df['time'][-1]
    tf = params['tf']
    wind = params['wind']
    qty_method = params['qty_method']
    std_1 = params.get('std_1', None)
    std_2 = params.get('std_2', None)

    dist_df = dist_df.select('time', 'ts', token_1, token_2, f'{token_1}_size', f'{token_2}_size',
             f'{token_1}_bid_price', f'{token_1}_ask_price', f'{token_2}_bid_price', f'{token_2}_ask_price',
             f'spread_mean_{wind}_{tf}', f'spread_std_{wind}_{tf}', f'z_score_{wind}_{tf}'
        ).rename({f"spread_mean_{wind}_{tf}": "spread_mean", 
                  f"spread_std_{wind}_{tf}": "spread_std", 
                  f"z_score_{wind}_{tf}": "z_score"})
    
    trades_dist = backtest(dist_df, token_1, token_2, dp_1, dp_2,
                -params['thresh_in'], -params['thresh_out'], params['thresh_in'], params['thresh_out'],
                long_possible=True, short_possible=True, dist_in=0, dist_out=0,
                balance=100, order_size=50, qty_method=qty_method, std_1=std_1, std_2=std_2,
                fee_rate=0.001, sl_std=5.0, sl_dist=1.0, sl_method='leave',
                leverage=leverage, verbose=2)
    # if trades_dist.height > 0:
    #     metrics_dist = analyze_strategy(trades_dist, start_date=valid_time, end_date=end_date, initial_balance=100.0)
    #     print(f'Max drawdown: {metrics_dist["max_drawdown"]}; profit_ratio: {metrics_dist["profit_ratio"]}')
    

In [ ]:
params = {'tf': '1h', 'wind': 120, 'thresh_in': 2.5, 'thresh_out': 0.25, 'qty_method': 'usdt_neutral',
         'std_1': std_1, 'std_2': std_2
         }

run_backtest(dist_df, params)

In [ ]:
def get_sensitivity(z_score_arr, profit_array):
    dz = np.diff(z_score_arr)
    dvu = np.diff(profit_array)
    
    mask = (np.abs(dz) >= 0.0001) & (np.abs(dvu) >= 0.0001)
    mask_pos = (np.abs(dz) >= 0.0001) & (np.abs(dvu) >= 0.0001) & (dz > 0)
    mask_neg = (np.abs(dz) >= 0.0001) & (np.abs(dvu) >= 0.0001) & (dz < 0)
    
    dvu_dz = dvu[mask] / dz[mask]
    dvu_dz_pos = dvu[mask_pos] / dz[mask_pos]
    dvu_dz_neg = dvu[mask_neg] / dz[mask_neg]

    return np.mean(dvu_dz), np.mean(dvu_dz_pos), np.mean(dvu_dz_neg)

def get_corr(z_score_array, profit_array, n_obs=1_000_00):
    z_arr = z_score_array[~np.isnan(z_score_array)][-n_obs:]
    pr_arr = profit_array[~np.isnan(profit_array)][-n_obs:]
    return np.corrcoef(z_arr, pr_arr)[0][1]

In [ ]:
token_1 = 'ARKM'
token_2 = 'MANTA'

tf = '1h'
wind = 96

test_in = 1.6
in_ = 2.2
out_ = 0.25
delete_dist = 0.2
fee_rate = 0.001
sl_std = 5.0

end_time = datetime.now().replace(tzinfo=ZoneInfo("Europe/Moscow"))
# valid_time = (end_time - timedelta(days=valid_length)).replace(hour=0, minute=0, second=0, microsecond=0)

valid_time = datetime(2025, 11, 27, 12, 33, 20, tzinfo=ZoneInfo('Europe/Moscow'))
# end_time   = datetime(2025, 11, 26, 22, 25, 55, tzinfo=ZoneInfo('Europe/Moscow'))
# start_time = valid_time - timedelta(days=train_length)
# start_ts   = int(datetime.timestamp(valid_time))

with open("./data/coin_information.pkl", "rb") as f:
    coin_information = pickle.load(f)

In [ ]:
valid_length = 3
train_length = 5
min_order = 42

df_1 = db_manager.get_tick_ob(token=token_1 + '_USDT',
                                     start_time=start_time,
                                     end_time=end_time)
df_2 = db_manager.get_tick_ob(token=token_2 + '_USDT',
                                     start_time=start_time,
                                     end_time=end_time)
df = make_df_from_orderbooks(df_1, df_2, token_1, token_2, start_time=start_time)
tick_df = make_df_from_orderbooks(df_1, df_2, token_1, token_2, start_time=start_time)

agg_df = make_trunc_df(df, timeframe=tf, token_1=token_1, token_2=token_2, method='triple')
dist_df = create_zscore_df(token_1, token_2, tick_df, agg_df, tf, np.array([wind]), min_order, start_ts, 
                             median_length=6, spr_method='dist')

In [ ]:
plt.figure(figsize=(15, 2))
plt.plot(dist_df['time'], dist_df[f'z_score_{wind}_{tf}'], c='blue', label='dist')
plt.axhline(y=0, color='black', linestyle='-', linewidth=2)
plt.axhline(y=-2.5, color='green', linestyle='--', linewidth=1)
plt.axhline(y=2.5, color='green', linestyle='--', linewidth=1)
plt.axhline(y=-0.25, color='red', linestyle='--', linewidth=1)
plt.axhline(y=0.25, color='red', linestyle='--', linewidth=1)
plt.grid();

In [ ]:
time_arr = dist_df['time'].to_numpy()
tss = dist_df['ts'].to_numpy()
price1 = dist_df[token_1].to_numpy()
price2 = dist_df[token_2].to_numpy()
bid_1 = dist_df[f"{token_1}_bid_price"].to_numpy()
ask_1 = dist_df[f"{token_1}_ask_price"].to_numpy()
bid_2 = dist_df[f"{token_2}_bid_price"].to_numpy()
ask_2 = dist_df[f"{token_2}_ask_price"].to_numpy()

hist_ts = agg_df['ts'].to_numpy()
hist_t1 = agg_df[token_1].to_numpy()
hist_t2 = agg_df[token_2].to_numpy()

In [ ]:
%%time
nrows = tss.shape[0]
ts_arr = np.full(nrows, np.nan)
z_score_arr = np.full(nrows, np.nan)
fixed_z_arr = np.full(nrows, np.nan)
profit_arr = np.full(nrows, np.nan)
spread_arr = np.full(nrows, np.nan)
corr_arr = np.full(nrows, np.nan)
mask = (np.abs(z_score_arr) >= 0.0001) & (np.abs(profit_arr) >= 0.0001)  & (np.abs(fixed_z_arr) >= 0.0001)

tracking_long = dict()
tracking_short = dict()

POS_NONE, POS_LONG, POS_SHORT = 0, 1, 2

pos_side = POS_NONE
side_1, side_2 = '', ''
qty_1, qty_2 = 0.0, 0.0
best_zscore = 0
test_price_1, test_price_2 = None, None
open_price_1, open_price_2 = None, None
fixed_mean, fixed_std = None, None
z_arr = []

for i in range(nrows):
    t1_price = price1[i]
    t2_price = price2[i]
    close_price_1 = ask_1[i] if side_1 == 'short' else bid_1[i]
    close_price_2 = ask_2[i] if side_1 == 'long' else bid_2[i]

    # Выберем из агрегированных цен только те, которые были до текущего момента
    mask = hist_ts < tss[i]
    t1_hist = hist_t1[mask][-wind:]
    t2_hist = hist_t2[mask][-wind:]

    # Сформируем массивы, в которых к историческим данным в конец добавим текущую медианную цену, и посчитаем z_score
    t1_arr_med = np.append(t1_hist, t1_price)
    t2_arr_med = np.append(t2_hist, t2_price)

    spread_mean, spread_std, zscore = get_dist_zscore(t1_arr_med, t2_arr_med, np.array([wind]))
    spread = np.log(t1_price) - np.log(t2_price)
    z_score = zscore[0]
    tm = datetime.fromtimestamp(tss[i])
        
    # ------------ Обработка входа в позицию ------------
    if pos_side == POS_NONE:
        # Если эта пара токенов ещё не отслеживается
        if z_score < -test_in and (token_1, token_2) not in tracking_long:
            print(f'{tm} Начинаем отслеживать long-пару {token_1} - {token_2}')
            
            fixed_mean, fixed_std = spread_mean[0], spread_std[0]
            print(f'{tm} Фиксируем среднее значение спреда {fixed_mean:.4f} и std {fixed_std:.4f}')
            test_price_1 = ask_1[i]
            test_price_2 = bid_2[i]
            side_1 = 'long'
            side_2 = 'short'
            qty_1, qty_2 = get_qty(token_1, token_2, test_price_1, test_price_2, None, coin_information, 
                       200, 0.001, None, None, method='usdt_neutral')
            fees = (qty_1 * test_price_1 + qty_2 * test_price_2) * fee_rate * 2
            
            tracking_long[(token_1, token_2)] = [[z_score], [-fees]]
        elif z_score > test_in and (token_1, token_2) not in tracking_short:
            print(f'{tm} Начинаем отслеживать short-пару {token_1} - {token_2}')

            fixed_mean, fixed_std = spread_mean[0], spread_std[0]
            print(f'{tm} Фиксируем среднее значение спреда {fixed_mean:.4f} и std {fixed_std:.4f}')
            
            test_price_1 = bid_1[i]
            test_price_2 = ask_2[i]
            side_1 = 'short'
            side_2 = 'long'
            qty_1, qty_2 = get_qty(token_1, token_2, test_price_1, test_price_2, None, coin_information, 
                       200, 0.001, None, None, method='usdt_neutral')
            fees = (qty_1 * test_price_1 + qty_2 * test_price_2) * fee_rate * 2
            tracking_short[(token_1, token_2)] = [[z_score], [-fees]]
            
        # Обновляем z_score и profit для потенциальной long-позиции
        if (token_1, token_2) in tracking_long:
            z_arr = tracking_long[(token_1, token_2)][0]
            best_zscore = min(z_arr)
            
            curr_profit_1 = calculate_profit(test_price_1, bid_1[i], qty_1, side_1)
            curr_profit_2 = calculate_profit(test_price_2, ask_2[i], qty_2, side_2)
            curr_profit = curr_profit_1 + curr_profit_2
            tracking_long[(token_1, token_2)][0].append(z_score)
            tracking_long[(token_1, token_2)][1].append(curr_profit)

            fixed_z_score = (spread - fixed_mean) / fixed_std
            fixed_z_arr[i] = fixed_z_score
            
            
            # Если z_score выходит из зоны входа в позицию, перестаём отслеживать пару
            if z_score > -test_in + delete_dist and pos_side == POS_NONE:
                print(f'{tm} Удаляем пару {token_1} - {token_2} из трекинга (z_score: {z_score:.3f})')
                curr_profit_1 = 0
                curr_profit_2 = 0
                curr_profit = 0
                tracking_long.pop((token_1, token_2))
                test_price_1, test_price_2 = None, None
                z_arr = []
                continue
            
            if z_score < -in_ and len(z_arr) > 120:
                print(f'{tm} Открываем лонг-позицию. z_score: {z_score:.3f}')

                z_score_array = np.array(z_arr)
                profit_array = np.array(tracking_long[(token_1, token_2)][1])
                
                corr_usdt = get_corr(z_score_array, profit_array, 120)
                print(f'Корреляция между движением z_score и profit: {corr_usdt:.2f}')

                dvu_dz, dvu_dz_pos, dvu_dz_neg = get_sensitivity(z_score_array, profit_array)
                print(f'Чуствительность: {dvu_dz:.3f}; pos: {dvu_dz_pos:.3f}; neg: {dvu_dz_neg:.3f}')

                open_price_1 = ask_1[i]
                open_price_2 = bid_2[i]
                side_1 = 'long'
                side_2 = 'short'
                qty_1, qty_2 = get_qty(token_1, token_2, open_price_1, open_price_2, None, coin_information, 
                           200, 0.001, None, None, method='usdt_neutral')

                fees = (qty_1 * open_price_1 + qty_2 * open_price_2) * fee_rate * 2
                pos_side = POS_LONG
                tracking_long.pop((token_1, token_2))
                z_score_array, profit_array, z_arr, pr_arr, corr_usdt = None, None, None, None, None
                test_price_1, test_price_2 = None, None
                z_arr = []
                continue
            
        # Обновляем z_score и profit для потенциальной short-позиции
        elif (token_1, token_2) in tracking_short:
            z_arr = tracking_short[(token_1, token_2)][0]
            best_zscore = max(z_arr)

            curr_profit_1 = calculate_profit(test_price_1, ask_1[i], qty_1, side_1)
            curr_profit_2 = calculate_profit(test_price_2, bid_2[i], qty_2, side_2)
            curr_profit = curr_profit_1 + curr_profit_2
            
            tracking_short[(token_1, token_2)][0].append(z_score)
            tracking_short[(token_1, token_2)][1].append(curr_profit)

            fixed_z_score = (spread - fixed_mean) / fixed_std
            fixed_z_arr[i] = fixed_z_score
            
            # Если z_score выходит из зоны входа в позицию, перестаём отслеживать пару
            if z_score < test_in - delete_dist and pos_side == POS_NONE:
                print(f'{tm} Удаляем пару {token_1} - {token_2} из трекинга (z_score: {z_score:.3f})')
                curr_profit_1 = 0
                curr_profit_2 = 0
                curr_profit = 0
                tracking_short.pop((token_1, token_2))
                test_price_1, test_price_2 = None, None
                z_arr = []
                continue
            
            if z_score > in_ and len(z_arr) > 120:
                print(f'{tm} Открываем шорт-позицию. z_score: {z_score:.3f}')
                
                z_score_array = np.array(tracking_short[(token_1, token_2)][0])
                profit_array = np.array(tracking_short[(token_1, token_2)][1])
                
                corr_usdt = get_corr(z_score_array, profit_array, 120)
                print(f'Корреляция между движением z_score и profit: {corr_usdt:.2f}')

                dvu_dz, dvu_dz_pos, dvu_dz_neg = get_sensitivity(z_score_array, profit_array)
                print(f'Чуствительность: {dvu_dz:.3f}; pos: {dvu_dz_pos:.3f}; neg: {dvu_dz_neg:.3f}')

                open_price_1 = bid_1[i]
                open_price_2 = ask_2[i]
                side_1 = 'short'
                side_2 = 'long'
                qty_1, qty_2 = get_qty(token_1, token_2, open_price_1, open_price_2, None, coin_information, 
                           200, 0.001, None, None, method='usdt_neutral')

                fees = (qty_1 * open_price_1 + qty_2 * open_price_2) * fee_rate * 2
                
                
                pos_side = POS_SHORT
                tracking_short.pop((token_1, token_2))
                z_score_array, profit_array, z_arr, pr_arr, corr_usdt = None, None, None, None, None
                test_price_1, test_price_2 = None, None
                z_arr = []
                continue
            
    
    # ------------ Обработка выхода из позиции ------------
    if pos_side == POS_LONG:
        curr_profit_1 = calculate_profit(open_price_1, bid_1[i], qty_1, side_1)
        curr_profit_2 = calculate_profit(open_price_2, ask_2[i], qty_2, side_2)
        curr_profit = curr_profit_1 + curr_profit_2

        z_score_arr[i] = z_score
        profit_arr[i] = curr_profit
        spread_arr[i] = spread
        
        fixed_z_score = (spread - fixed_mean) / fixed_std
        fixed_z_arr[i] = fixed_z_score

        # Сохраняем текущую корреляцию
        zsc = z_score_arr[~np.isnan(z_score_arr)]
        pr = spread_arr[~np.isnan(spread_arr)]
        corr_arr[i] = np.corrcoef(zsc, pr)[0][1]
        
        # Проверяем стоп-лосс
        # if abs(fixed_z_score) > sl_std:
        #     print(f'{tm} STOP-LOSS! Закрываем long-позицию. z_score: {z_score:.3f}, fixed z_score: {fixed_z_score:.3f}, profit: {curr_profit:.2f}')
        #     print(f'fixed_z_score: {fixed_z_score}; spread: {spread}; mean: {fixed_mean} std: {fixed_std} на шаге {i}')
        #     break
        
        if z_score > out_:
            print(f'{tm} Закрываем long-позицию. z_score: {z_score:.3f}, profit: {curr_profit:.2f}')
            pos_side = POS_NONE
            open_price_1, open_price_2, qty_1, qty_2 = None, None, None, None
            fixed_mean, fixed_std = None, None
            side_1, side_2 = '', ''
            continue
    
    if pos_side == POS_SHORT:
        curr_profit_1 = calculate_profit(open_price_1, ask_1[i], qty_1, side_1)
        curr_profit_2 = calculate_profit(open_price_2, bid_2[i], qty_2, side_2)
        curr_profit = curr_profit_1 + curr_profit_2
        
        z_score_arr[i] = z_score
        profit_arr[i] = curr_profit
        spread_arr[i] = spread
        
        fixed_z_score = (spread - fixed_mean) / fixed_std
        fixed_z_arr[i] = fixed_z_score
        
        # Сохраняем текущую корреляцию
        zsc = z_score_arr[~np.isnan(z_score_arr)]
        pr = spread_arr[~np.isnan(spread_arr)]
        corr_arr[i] = np.corrcoef(zsc, pr)[0][1]

        # Проверяем стоп-лосс
        # if abs(fixed_z_score) > sl_std:
        #     print(f'{tm} STOP-LOSS! Закрываем short-позицию. z_score: {z_score:.3f}, fixed z_score: {fixed_z_score:.3f}, profit: {curr_profit:.2f}')
        #     print(f'fixed_z_score: {fixed_z_score}; spread: {spread}; mean: {fixed_mean} std: {fixed_std} на шаге {i}')
        
        if z_score < -out_:
            print(f'{tm} Закрываем шорт-позицию. z_score: {z_score:.3f}, profit: {curr_profit:.2f}')
            pos_side = POS_NONE
            open_price_1, open_price_2, qty_1, qty_2 = None, None, None, None
            fixed_mean, fixed_std = None, None
            side_1, side_2 = '', ''
            continue


In [ ]:
np.corrcoef(zsc, pr)[0][1]

In [ ]:
mask = (np.abs(z_score_arr) >= 0.0001) & (np.abs(profit_arr) >= 0.0001)  & (np.abs(fixed_z_arr) >= 0.0001)
zsc = z_score_arr[mask]
spr = spread_arr[mask]
pr = profit_arr[mask]
fxz = fixed_z_arr[mask]
cr = corr_arr[~np.isnan(corr_arr)]

dz = np.diff(zsc)
dpr = np.diff(pr)
eps = 0.0000001
mask = (np.abs(dz) >= eps) & (np.abs(dpr) >= eps)
mask_pos = (np.abs(dz) >= eps) & (np.abs(dpr) >= eps) & (dz > 0)
mask_neg = (np.abs(dz) >= eps) & (np.abs(dpr) >= eps) & (dz < 0)

dpr_dz = dpr[mask] / dz[mask]
dpr_dz_pos = dpr[mask_pos] / dz[mask_pos]
dpr_dz_neg = dpr[mask_neg] / dz[mask_neg]

avg_dpr_dz = moving_average(dpr_dz, n=120)
avg_pos_dz = moving_average(dpr_dz_pos, n=120)
avg_neg_dz = moving_average(dpr_dz_neg, n=120)

In [ ]:
def plot_spread():
    x = range(len(zsc))

    fig, (ax1, ax2, ax3, ax4) = plt.subplots(nrows=4, ncols=1, figsize=(14, 8))
    ax1.plot(x, zsc, c='blue', label='z_score')
    ax1.plot(x, fxz, c='gray', label='fixed z_score')
    ax1.axhline(y=0, color='black', linestyle='-', linewidth=1)
    ax1.axhline(y=-2.5, color='green', linestyle='--', linewidth=1)
    ax1.axhline(y=2.5, color='green', linestyle='--', linewidth=1)
    # ax1.axhline(y=-0.25, color='red', linestyle='--', linewidth=1)
    # ax1.axhline(y=0.25, color='red', linestyle='--', linewidth=1)
    ax1.set_title('Profit & z_score')
    ax1.grid();
    ax1.legend()
    
    ax5 = ax1.twinx()
    ax5.plot(x, pr, c='red', label='profit')
    
    ax2.set_title('Sensitivity')
    ax2.plot(range(len(avg_pos_dz)), avg_pos_dz, c='green', label='positive')
    ax2.plot(range(len(avg_neg_dz)), avg_neg_dz, c='red', label='negative')
    ax2.legend()
    # ax2.set_ylim(-50, 50)
    ax2.grid();
    
    ax3.plot(x, spr)
    ax3.set_title('Spread')
    ax3.grid()

    ax4.plot(range(len(cr)), cr, c='green', label='corr', linewidth=2)
    ax4.set_title('Correlation')
    ax4.set_ylim(-1, 1)
    ax4.grid()
    
    fig.tight_layout()

In [ ]:
plot_spread()

In [ ]:
cr.min()

In [ ]:
corr_usdt = np.corrcoef(zsc, pr)[0][1]
corr_usdt

In [ ]:
np.mean(dpr_dz), np.mean(dpr_dz_pos), np.mean(dpr_dz_neg)

In [ ]:
len(dpr_dz_pos),len(dpr_dz_neg)

In [ ]:
len(dpr_dz_pos) * np.mean(dpr_dz_pos) - len(dpr_dz_neg) * np.mean(dpr_dz_neg)

In [ ]:
dpr_dz_pos.sum(), -dpr_dz_neg.sum()

In [ ]:
zsc

In [ ]:
start, stop = 240, 360
round(float(dz[start:stop].sum()), 4), round(float(dpr[start:stop].sum()), 4)

In [ ]:
0.3311 / 0.193, -0.7224 / -0.4186, 0.0645/0.0649

In [ ]:
fig, ax1 = plt.subplots(figsize=(4, 4))
ax1.scatter(dz, dpr)
ax1.grid();

In [ ]:
def moving_average(a, n=3):
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

In [ ]:
len(z_score_array), len(profit_array)

In [ ]:
n = 50

fig, ax1 = plt.subplots(1, 1, figsize=(15, 3))
ax1.plot(range(n), z_score_array[-n:], label=f'z_score', c='blue')
ax1.set_ylabel('z_score')
ax2 = ax1.twinx()
ax2.plot(range(n), profit_array[-n:], label=f'profit', c='green')
ax2.set_ylabel('usdt')
plt.tight_layout()

In [ ]:
ARKM_USDT"	"MANTA_USDT"	2025-11-27 12:33:20

In [7]:
token_1 = 'ARKM'
token_2 = 'MANTA'

open_time = datetime(2025, 11, 27, 12, 33, 20, tzinfo=ZoneInfo('Europe/Moscow'))
close_time = datetime.now()
open_ts = int(datetime.timestamp(open_time))
close_ts = int(datetime.timestamp(close_time))

z_score_hist = db_manager.get_zscore_history(token_1 + '_USDT', token_2 + '_USDT', open_ts, close_ts)

ComputeError: could not append value: 1.798098 of type: f64 to the builder; make sure that all rows have the same schema or consider increasing `infer_schema_length`

it might also be that a value overflows the data-type's capacity

In [5]:
int(datetime.timestamp(datetime(2025, 11, 27, 13, 55, tzinfo=ZoneInfo('Europe/Moscow'))))

1764240900

In [ ]:
z_score_hist

In [ ]:
fig, ax1 = plt.subplots(figsize=(14, 3))
ax1.plot(dist_df['time'], dist_df[f'z_score_{wind}_{tf}'], c='blue', label='calc_z_score')
ax1.plot(z_score_hist['time'], z_score_hist['z_score'], c='gray', label='real_z_score')
ax1.axhline(y=0, color='black', linestyle='-', linewidth=2)
ax1.axhline(y=-2.5, color='green', linestyle='--', linewidth=1)
ax1.axhline(y=2.5, color='green', linestyle='--', linewidth=1)
ax1.axhline(y=-0.25, color='red', linestyle='--', linewidth=1)
ax1.axhline(y=0.25, color='red', linestyle='--', linewidth=1)

ax2 = ax1.twinx()
ax2.plot(z_score_hist['time'], z_score_hist['profit'], c='red', label='profit')

ax1.grid();

In [ ]:
z_score_hist.select('z_score','profit').corr()

In [ ]:
z_score_hist = z_score_hist.with_columns(
    pl.rolling_corr(
        a="z_score",         # Первый столбец
        b="profit",          # Второй столбец
        window_size=720,     # Размер скользящего окна
        ddof=1               # Степени свободы для дисперсии
    ).alias("rolling_corr")
)

In [ ]:
fig, ax1 = plt.subplots(figsize=(14, 3))
ax1.plot(z_score_hist['time'], z_score_hist['rolling_corr'])
ax1.grid();

# ax2 = ax1.twinx()
# ax2.plot(range(len(r_corr)), r_corr, c='red', label='numpy')

In [ ]:
def rolling_corr_numpy(x, y, window):
    n = len(x)
    corr = np.full(n, np.nan)
    
    # Предварительные вычисления для оптимизации
    x_rolling = np.lib.stride_tricks.sliding_window_view(x, window)
    y_rolling = np.lib.stride_tricks.sliding_window_view(y, window)
    
    # Векторизованный расчет корреляции
    for i in range(len(x_rolling)):
        idx = i + window - 1
        corr[idx] = np.corrcoef(x_rolling[i], y_rolling[i])[0, 1]
    
    return corr

In [ ]:
r_corr = rolling_corr_numpy(zsc, pr, 360)

In [ ]:
std_1_long = 0.009992
std_2_long = 0.016839


qty_1_vol_short, qty_2_vol_short = get_qty(token_1, token_2, open_price_1, open_price_2, None, coin_information, 
                       200, 0.001, t1_open.std(), t2_open.std(), method='vol_neutral')
qty_1_vol_long, qty_2_vol_long = get_qty(token_1, token_2, open_price_1, open_price_2, None, coin_information, 
                       200, 0.001, std_1_long, std_2_long, method='vol_neutral')

In [ ]:
qty_1, qty_2, qty_1_vol_short, qty_2_vol_short, qty_1_vol_long, qty_2_vol_long

In [ ]:
z_arr = z_score_arr[~np.isnan(z_score_arr)]
pr_arr = profit_usdt_arr[~np.isnan(profit_usdt_arr)]
vnl_arr = profit_vnl_arr[~np.isnan(profit_vnl_arr)]
vns_arr = profit_vns_arr[~np.isnan(profit_vns_arr)]

corr_usdt = np.corrcoef(z_arr, pr_arr)[0][1]
corr_vln = np.corrcoef(z_arr, vnl_arr)[0][1]
corr_vsn = np.corrcoef(z_arr, vns_arr)[0][1]
print(f'Corr. Usdt: {corr_usdt:.2f}; Vol-neut short: {corr_vsn:.2f}; Vol-neut long: {corr_vln:.2f}')

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(15, 3), sharex=True)
ax1.plot(time_arr, z_score_arr, c='blue', label='z_score')

ax2 = ax1.twinx()
ax2.plot(time_arr, profit_usdt_arr, c='green', label='usdt_neutral')
ax2.plot(time_arr, profit_vnl_arr, c='red', label='vol_long_neutral')
ax2.plot(time_arr, profit_vns_arr, c='magenta', label='vol_short_neutral')

fig.legend()
plt.grid()

In [ ]:
params = {'tf': '1h', 'wind': 96, 'thresh_in': 2.25, 'thresh_out': 0.25, 'dist_in': 0.0, 'dist_out': 0.0}

trades_df = backtest(dist_df.rename({f'z_score_{params["wind"]}_{params["tf"]}': 'z_score'}), token_1, token_2, dp_1, dp_2, ps_1, ps_2,
                -params['thresh_in'], -params['thresh_out'], params['thresh_in'], params['thresh_out'],
                long_possible=True, short_possible=True, dist_in=params['dist_in'], dist_out=params['dist_out'],
                balance=100, order_size=50, fee_rate=0.00055, stop_loss_std=8.0, sl_method='leave',
                sl_seconds = 60, leverage=leverage, verbose=2)

#### Анализ совершённых сделок

In [ ]:
orders = pl.read_ndjson('./logs/trades.jsonl')

token_1 = 'APT'
token_2 = 'FIL'

ot = "2025-09-16 17:07:14"
ct = "2025-09-16 23:09:17"

open_ = orders.filter((pl.col('token_1') == token_1) & (pl.col('token_2') == token_2) & (pl.col('action') == 'open') & (pl.col('ct') == ot))
close = orders.filter((pl.col('token_1') == token_1) & (pl.col('token_2') == token_2) & (pl.col('action') == 'close') & (pl.col('ct') == ct))

In [ ]:
open_.select('ct', 'token_1', 'token_2', 'tf', 'wind', 'thresh_in', 'thresh_out', 'side', 'action',
            't1_bid_price', 't1_ask_price', 't2_bid_price', 't2_ask_price', 'z_score', 'beta')

In [ ]:
close.select('ct', 'token_1', 'token_2', 'tf', 'wind', 'thresh_in', 'thresh_out', 'side', 'action',
            't1_bid_price', 't1_ask_price', 't2_bid_price', 't2_ask_price', 'z_score', 'beta')

In [ ]:
max_position_size = 200
leverage = 2
tf = open_['tf'][0]
wind = open_['wind'][0]

side = open_['side'][0]
side_2 = 'short' if side == 'long' else 'long'
beta = open_['beta'][0]
t1_open = open_['t1'].to_numpy()[0]
t2_open = open_['t2'].to_numpy()[0]
t1_close = close['t1'].to_numpy()[0]
t2_close = close['t2'].to_numpy()[0]

t1_op = open_['t1_ask_price'][0] if side == 'long' else open_['t1_bid_price'][0]
t2_op = open_['t2_bid_price'][0] if side == 'long' else open_['t2_ask_price'][0]
t1_cl = close['t1_bid_price'][0] if side == 'long' else close['t1_ask_price'][0]
t2_cl = close['t2_ask_price'][0] if side == 'long' else close['t2_bid_price'][0]

In [ ]:
make_zscore_df(pl.DataFrame({token_1: t1_open, token_2: t2_open}), token_1, token_2, wind, method='lr').tail(2)

In [ ]:
train_len = 80
wind = 10
winds = np.array((wind,))
tf = '4h'
median_length = 6
min_order = 50

train_time = datetime(2025, 9, 12, 17, 0, tzinfo=ZoneInfo("Europe/Moscow"))
start_time = datetime(2025, 9, 16, 17, 7, 14, tzinfo=ZoneInfo("Europe/Moscow"))
end_time = datetime(2025, 9, 16, 23, 9, 17, tzinfo=ZoneInfo("Europe/Moscow"))

start_ts = int(datetime.timestamp(start_time))

df_1 = db_manager.get_tick_ob(token=token_1 + '_USDT',
                                 start_time=train_time,
                                 end_time=end_time)
df_2 = db_manager.get_tick_ob(token=token_2 + '_USDT',
                                 start_time=train_time,
                                 end_time=end_time)

avg_df = make_df_from_orderbooks(df_1, df_2, token_1, token_2, start_time=train_time)
agg_df = make_trunc_df(avg_df, timeframe='4h', token_1=token_1, token_2=token_2, method='triple', offset='3h')

df_sec = make_trunc_df(avg_df, timeframe='1s', token_1=token_1, token_2=token_2,
                       start_date=start_time - timedelta(seconds=6),
                       end_date = end_time + timedelta(seconds=6),
                       method='last', return_bid_ask=True)
spread_df = create_zscore_df(token_1, token_2, df_sec, agg_df, winds, min_order, start_ts, median_length)

df = spread_df.select('time', 'ts', token_1, token_2, f'{token_1}_size', f'{token_2}_size',
     f'{token_1}_bid_price', f'{token_1}_ask_price', f'{token_1}_bid_size', f'{token_1}_ask_size',
     f'{token_2}_bid_price', f'{token_2}_ask_price', f'{token_2}_bid_size', f'{token_2}_ask_size',
     'z_score').filter(
        (pl.col('time') >= start_time) & (pl.col('time') <= end_time)
     )

In [ ]:
spread_df = create_zscore_df(token_1, token_2, df_sec, agg_df, winds, min_order, start_ts, median_length)


In [ ]:
t1_op, t1_cl

In [ ]:
t2_op, t2_cl

In [ ]:
qty_1, qty_2 = get_qty(token_1, token_2, t1_op, t2_op, beta, coin_information, max_position_size * leverage,
                          method='usdt_neutral')
qty_1, qty_2

In [ ]:
pr_1 = calculate_profit(open_price=t1_op, close_price=t1_cl, n_coins=qty_1, side=side)
pr_2 = calculate_profit(open_price=t2_op, close_price=t2_cl, n_coins=qty_2, side=side_2)
pr_1, pr_2, pr_1 + pr_2

In [ ]:
t1_op, t1_cp

In [ ]:
make_zscore_df(pl.DataFrame({token_1: t1_close, token_2: t2_close}), token_1, token_2, wind, method='lr').tail(1)

In [ ]:
qty_1 = 10881
qty_2 = 4375

open_spread = -0.000066
close_spread = 0.000021
open_mean = 0.000015
open_std = 0.00005

t1_bid_ask_spread = open_['t1_ask_price'][0] - open_['t1_bid_price'][0]
t2_bid_ask_spread = open_['t2_ask_price'][0] - open_['t2_bid_price'][0]
beta = open_['beta'][0]
z_score = open_['z_score'][0]
fee_rate = 0.00055

In [ ]:
# Доход за 1 стандартное отклонение
profit_per_std = qty_1 * open_std
profit_per_std

In [ ]:
# Считаем профит
spread_profit = abs(open_spread - (open_mean + 0.5 * open_std))
real_spread_profit = (close_spread - open_spread) * qty_1
spread_profit * qty_1, real_spread_profit

In [ ]:
# Комиссия за 4 сделки, каждая на 200$ (100$ с плечом 2)
fees = 4 * 200 * fee_rate

# Bid-ask spread
bid_ask_slippage = t1_bid_ask_spread * qty_1 + t2_bid_ask_spread * qty_2
fees, bid_ask_slippage

#### Обработка больших датафреймов

In [ ]:
from tqdm.notebook import tqdm
import polars as pl
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
from bot.utils.files import load_config

config = load_config('./bot/config/config.yaml')

method = config['spr_method']
start_time = config['valid_time']
end_time = config['end_time']

In [ ]:
token_pairs = []
with open('./bot/config/token_pairs.txt', 'r') as file:
    for line in file:
        a, b = line.strip().split()
        token_pairs.append((a, b))
len(token_pairs)

In [ ]:
# --- Создание объединённого датафрейма из нескольких ---
tfs = ('1h', )
method = 'dist'

time_series = pl.datetime_range(start=start_time, end=end_time, interval="5s", eager=True)
main_df = pl.DataFrame({'time': time_series})

for token_1, token_2 in tqdm(token_pairs):
    try:
        spread_df = pl.read_parquet(f'./data/pair_backtest/{token_1}_{token_2}_1h_{method}.parquet', 
                low_memory=True, rechunk=True, use_pyarrow=True).filter(
                (pl.col('time') >= start_time) & (pl.col('time') < end_time)
            )

        cols_to_rn = [col for col in spread_df.columns for tf in tfs if col.endswith(tf)]
        prefix = f"{token_1}_{token_2}_"
        mapping = {c: f"{prefix}{c}" for c in cols_to_rn}
        spread_df = spread_df.rename(mapping)

        cols_to_drop = [col for col in spread_df.columns if col in main_df.columns]
        cols_to_drop.remove('time')
        spread_df = spread_df.drop(cols_to_drop)

        main_df = main_df.join(spread_df, on='time', how='full', coalesce=True)
    except FileNotFoundError:
        print(f'FileNotFoundError: {token_1} - {token_2}')
        continue

In [ ]:
spr_cols = [col for col in main_df.columns if 'spread' in col]
main_df.drop(spr_cols).tail(3)

In [ ]:
main_df.tail(10)

In [ ]:
main_df[1:-1].drop(spr_cols).write_parquet('./data/train_data.parquet')

In [ ]:
# --- Код для склеивания по частям двух датафреймов, записанных на диске, в один ---
import polars as pl

df_1 = pl.read_parquet('./data/test_1.parquet', low_memory=True, use_pyarrow=True)
df_2 = pl.read_parquet('./data/test_2.parquet', low_memory=True, use_pyarrow=True)
print(df_1.shape, df_2.shape)

full_df = df_1.join(df_2[:, :500], on='time', how='full', coalesce=True)
full_df.write_parquet(f'./data/full_1.parquet')

cols_to_drop = df_2[:, :500].columns
cols_to_drop.remove('time')
df_2 = df_2.drop(cols_to_drop)
print(full_df.shape, df_2.shape)

df_2.write_parquet(f'./data/full_2.parquet')

In [ ]:
token_1 = 'JUP'
token_2 = 'MANA'
method = 'dist'

In [ ]:
pl.read_parquet(f'./data/pair_backtest/{token_1}_{token_2}_{method}_full.parquet', 
                low_memory=True, rechunk=True, use_pyarrow=True)